# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:////home/workspace/DisasterResponse.db')
df = pd.read_sql_table('MessagesWithCatTable', engine)
X = df['message']
Y = df.iloc[:,4:]

In [4]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [4]:
import nltk
nltk.download(['punkt', 'wordnet'])
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [28]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(max_df = 0.5, ngram_range = (1, 1), tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=50), n_jobs=-1))
])


'features__text_pipeline__vect__ngram_range': (1, 1)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1))])

In [30]:
# predict on test data
Y_pred = pipeline.predict(X_test)

In [17]:
Y_test.shape

(6554, 36)

In [19]:
Y_pred.shape

(6554, 36)

In [31]:
index = [str(x) for x in list(Y_test.index)]
category_names = list(Y.columns)
Y_pred = pd.DataFrame(Y_pred, index= index , columns = category_names )

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [32]:
print(classification_report(Y_test, Y_pred, target_names= category_names))

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5012
               request       0.87      0.42      0.56      1131
                 offer       0.00      0.00      0.00        25
           aid_related       0.78      0.62      0.69      2737
          medical_help       0.73      0.03      0.06       559
      medical_products       0.78      0.05      0.09       366
     search_and_rescue       0.45      0.03      0.05       189
              security       0.00      0.00      0.00       115
              military       0.60      0.06      0.11       206
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.25      0.39       374
                  food       0.87      0.39      0.54       759
               shelter       0.82      0.26      0.39       552
              clothing       0.71      0.11      0.18        95
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [14]:
pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_... smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('starting_verb', StartingVerbExtractor())],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_s

In [41]:
parameters = {
    'clf__estimator__n_estimators': [10],
    'features__text_pipeline__vect__ngram_range': [(1, 1)]
}
cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_macro', cv=None, n_jobs=-1,verbose=10) 
cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)
print("\nBest Parameters:", cv.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__n_estimators=10, features__text_pipeline__vect__ngram_range=(1, 1) 


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=10, features__text_pipeline__vect__ngram_range=(1, 1), score=0.19331399108921088, total= 1.7min
[CV] clf__estimator__n_estimators=10, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.4min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=10, features__text_pipeline__vect__ngram_range=(1, 1), score=0.19824642794650776, total= 1.7min
[CV] clf__estimator__n_estimators=10, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  4.8min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=10, features__text_pipeline__vect__ngram_range=(1, 1), score=0.202840975546375, total= 1.6min


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  7.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  7.1min finished



Best Parameters: {'clf__estimator__n_estimators': 10, 'features__text_pipeline__vect__ngram_range': (1, 1)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

The tuned model has parameters n= 10 for the RandomForestClassifier and max_df = 0.5 for the CountVectorizer feature extraction method. Here's the accuracy, precision and recall of the tuned model:

In [42]:
print(classification_report(Y_test, Y_pred, target_names= category_names))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5012
               request       0.81      0.38      0.51      1131
                 offer       0.00      0.00      0.00        25
           aid_related       0.74      0.55      0.63      2737
          medical_help       0.60      0.06      0.11       559
      medical_products       0.68      0.06      0.11       366
     search_and_rescue       0.52      0.06      0.11       189
              security       0.67      0.02      0.03       115
              military       0.43      0.08      0.13       206
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.25      0.39       374
                  food       0.81      0.27      0.41       759
               shelter       0.82      0.14      0.25       552
              clothing       0.60      0.06      0.11        95
                 money       0.75      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

We tried a K-neighbors classifier with K=8, but the performance was poorer than the Random Forest Classifier. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
pipeline2 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(KNeighborsClassifier(n_neighbors=8), n_jobs=-1))
])



X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train,Y_train)

Y_pred = pipeline2.predict(X_test)
index = [str(x) for x in list(Y_test.index)]
category_names = list(Y.columns)
Y_pred = pd.DataFrame(Y_pred, index= index , columns = category_names )

print(classification_report(Y_test, Y_pred, target_names= category_names))

### 9. Export your model as a pickle file

In [43]:
import pickle
pickle.dump(cv, open('train_classifier.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.